In [22]:
# from sqlalchemy import create_engine
import pandas as pd
import numpy as np
# from binance import Client
import matplotlib.pyplot as plt
# import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import sys
import ta
import yaml
from data import get_data
import os
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/backtesting')
from backtest import * 
from strategies import *
from config import *
# color_pal = sns.color_palette()
# plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    f1_score,
    mean_squared_error,
    accuracy_score,
    r2_score,
    mean_absolute_error,
)

In [4]:
FEATURES = ['Open',
    'High',
    'Low',
    'Volume',
    'RSI_2',
    'SMA_20',
    'RSI_4',
    'SMA_40',
    'RSI_6',
    'SMA_60',
    'RSI_8',
    'SMA_80']
TARGET = 'Target'

In [8]:
df = pd.read_csv('data.csv')
df.set_index("Time", inplace=True)
df = pd.read_csv('data.csv')

In [20]:
df.shape

(59921, 15)

In [17]:
tss = TimeSeriesSplit(n_splits=5, test_size=3*24*60)
df = df.sort_index()

fold = 0
preds = []
scores = []

for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    clf = lgb.LGBMClassifier(n_estimators=1000)
    clf.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = clf.predict(X_test)
    preds.append(y_pred)
    score = f1_score(y_test, y_pred)
    scores.append(score)

[100]	training's binary_logloss: 0.652832	valid_1's binary_logloss: 0.695833
[200]	training's binary_logloss: 0.62319	valid_1's binary_logloss: 0.698619
[300]	training's binary_logloss: 0.597564	valid_1's binary_logloss: 0.702441
[400]	training's binary_logloss: 0.574426	valid_1's binary_logloss: 0.706435
[500]	training's binary_logloss: 0.553463	valid_1's binary_logloss: 0.711089
[600]	training's binary_logloss: 0.532816	valid_1's binary_logloss: 0.714488
[700]	training's binary_logloss: 0.514628	valid_1's binary_logloss: 0.717471
[800]	training's binary_logloss: 0.496315	valid_1's binary_logloss: 0.719973
[900]	training's binary_logloss: 0.480044	valid_1's binary_logloss: 0.722234
[1000]	training's binary_logloss: 0.465002	valid_1's binary_logloss: 0.725299
[100]	training's binary_logloss: 0.655826	valid_1's binary_logloss: 0.697673
[200]	training's binary_logloss: 0.628066	valid_1's binary_logloss: 0.702078
[300]	training's binary_logloss: 0.603626	valid_1's binary_logloss: 0.704248

In [18]:
print(f'score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

RMSE score across folds 0.4996
Fold scores:[0.5188419117647058, 0.5426714195848187, 0.48532948532948533, 0.4695270599707459, 0.4814723926380368]


In [ ]:
reg = xgb.XGBRegressor(booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
reg.fit(data[FEATURES], data[TARGET],
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)